<a href="https://colab.research.google.com/github/Witcape/3D_Printing/blob/main/Inceptionnet_SRCNN_FINAL_FINAL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
import os

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
train_dir = "/content/drive/MyDrive/Final_Final/Train"
test_dir = "/content/drive/MyDrive/Final_Final/Test"

# List all image paths and corresponding labels
image_paths = []
labels = []

for class_dir in os.listdir(train_dir):
    class_path = os.path.join(train_dir, class_dir)

    if not os.path.isdir(class_path):
        continue

    for img in os.listdir(class_path):
        image_paths.append(os.path.join(class_path, img))
        labels.append(class_dir)  # Use the directory name as the label

# Split into train and validation
train_paths, val_paths, train_labels, val_labels = train_test_split(
    image_paths, labels, test_size=0.2, stratify=labels, random_state=42
)


In [4]:
unique_labels = set(labels)
unique_labels
import pandas as pd

In [5]:
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)
# Create DataFrame for training data
train_df = pd.DataFrame({
    "filepath": train_paths,
    "class": train_labels
})

# Create DataFrame for validation data
val_df = pd.DataFrame({
    "filepath": val_paths,
    "class": val_labels
})



val_datagen = ImageDataGenerator(rescale=1.0/255)

train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    x_col="filepath",
    y_col="class",
    target_size=(299, 299),  # Image size for InceptionV3
    batch_size=32,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_dataframe(
    dataframe=val_df,
    x_col="filepath",
    y_col="class",
    target_size=(299, 299),
    batch_size=32,
    class_mode='categorical'
)

Found 692 validated image filenames belonging to 3 classes.
Found 174 validated image filenames belonging to 3 classes.


/usr/local/lib/python3.10/dist-packages/keras/src/legacy/preprocessing/image.py:920: UserWarning: Found 3 invalid image filename(s) in x_col="filepath". These filename(s) will be ignored.
  warnings.warn(


In [6]:
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(299, 299, 3))

87910968/87910968 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step


In [7]:
x = base_model.output
x = GlobalAveragePooling2D()(x)  # Global average pooling
x = Dense(1024, activation='relu')(x)  # Fully connected layer
predictions = Dense(len(train_generator.class_indices), activation='softmax')(x)  # Output layer

model = Model(inputs=base_model.input, outputs=predictions)

In [8]:
for layer in base_model.layers:
    layer.trainable = False

In [9]:
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [11]:
history_fine = model.fit(
    train_generator,
    epochs=10,
    validation_data=val_generator
)

Epoch 1/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 20s 695ms/step - accuracy: 0.7904 - loss: 0.5148 - val_accuracy: 0.8333 - val_loss: 0.4835
Epoch 2/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 21s 632ms/step - accuracy: 0.8660 - loss: 0.3551 - val_accuracy: 0.8218 - val_loss: 0.4837
Epoch 3/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 20s 667ms/step - accuracy: 0.8521 - loss: 0.3970 - val_accuracy: 0.7759 - val_loss: 0.5230
Epoch 4/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 21s 686ms/step - accuracy: 0.8945 - loss: 0.3243 - val_accuracy: 0.8621 - val_loss: 0.4242
Epoch 5/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 40s 631ms/step - accuracy: 0.8844 - loss: 0.3063 - val_accuracy: 0.8678 - val_loss: 0.4021
Epoch 6/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 41s 642ms/step - accuracy: 0.9044 - loss: 0.2811 - val_accuracy: 0.8161 - val_loss: 0.4756
Epoch 7/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 20s 671ms/step - accuracy: 0.8919 - loss: 0.2882 - val_accuracy: 0.8218 - val_loss: 0.5164
Epoch 8/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 21s 677ms/step - accuracy: 0.8941 - loss: 0.2658 - val_accu

In [12]:
for layer in base_model.layers[-50:]:
    layer.trainable = True

In [13]:
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [15]:
history_fine = model.fit(
    train_generator,
    epochs=10,
    validation_data=val_generator
)

Epoch 1/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 25s 918ms/step - accuracy: 0.9608 - loss: 0.1256 - val_accuracy: 0.8391 - val_loss: 0.4356
Epoch 2/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 37s 694ms/step - accuracy: 0.9651 - loss: 0.0965 - val_accuracy: 0.8276 - val_loss: 0.4678
Epoch 3/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 41s 650ms/step - accuracy: 0.9640 - loss: 0.0976 - val_accuracy: 0.9138 - val_loss: 0.3377
Epoch 4/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 42s 676ms/step - accuracy: 0.9851 - loss: 0.0479 - val_accuracy: 0.8851 - val_loss: 0.3956
Epoch 5/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 20s 681ms/step - accuracy: 0.9891 - loss: 0.0357 - val_accuracy: 0.9253 - val_loss: 0.3231
Epoch 6/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 21s 638ms/step - accuracy: 0.9713 - loss: 0.0615 - val_accuracy: 0.9080 - val_loss: 0.3125
Epoch 7/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 42s 678ms/step - accuracy: 0.9931 - loss: 0.0279 - val_accuracy: 0.9310 - val_loss: 0.2675
Epoch 8/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 41s 677ms/step - accuracy: 0.9947 - loss: 0.0189 - val_accu

In [16]:
test_datagen = ImageDataGenerator(rescale=1.0/255)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(299, 299),
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)

test_loss, test_acc = model.evaluate(test_generator)
print(f"Test Accuracy: {test_acc:.2f}")

Found 387 images belonging to 3 classes.
13/13 ━━━━━━━━━━━━━━━━━━━━ 150s 12s/step - accuracy: 0.9168 - loss: 0.2879
Test Accuracy: 0.90
